In [7]:
# импорт основных классов
import numpy as np
import json
import random
import math
from pip._internal import main
main(['install', 'termcolor','random'])
from termcolor import colored, cprint

  Could not find a version that satisfies the requirement random (from versions: )
No matching distribution found for random


In [8]:
#создаем класс, который генерит крестики и нолики
class createXO(object):
    
    def __init__(self):
        self.matrix = [[0]*7 for i in range(7)]
        pass
    
    def createX(self):
      # задаем начальную точку для рисования крестика
        x = random.randint(2,4)
        y = random.randint(2,4)
        self.matrix[x][y] = 1
        # задаем радиус рисования крестика
        rad = min(6-x,6-y, x, y)
        # заполняем крестик по диагоналям
        for i in range(1, rad):
            self.matrix[x-i][y-i] = 1
            self.matrix[x-i][y+i] = 1
            self.matrix[x+i][y-i] = 1
            self.matrix[x+i][y+i] = 1
        # оставшиеся клетки заполняем случайно 0/1
        self.matrix[x+rad][y-rad] = random.randint(0,1)
        self.matrix[x-rad][y+rad] = random.randint(0,1)
        self.matrix[x+rad][y+rad] = random.randint(0,1)
        self.matrix[x-rad][y-rad] = random.randint(0,1)
        return self.matrix
    
    def createO(self):
        x = random.randint(2,4)
        y = random.randint(2,4)
        rad = min(6-x,6-y, x, y)
        for i in range(-rad+1, rad):
            self.matrix[x+i][y-rad] = 1
            self.matrix[x+i][y+rad] = 1
            self.matrix[x+rad][y+i] = 1
            self.matrix[x-rad][y+i] = 1
        self.matrix[x+rad][y-rad] = random.randint(0,1)
        self.matrix[x-rad][y+rad] = random.randint(0,1)
        self.matrix[x+rad][y+rad] = random.randint(0,1)
        self.matrix[x-rad][y-rad] = random.randint(0,1)
        return self.matrix
    def lenCreate(self):
        return len(self.matrix)
    
    def getIndex(self,i,j):
        return self.matrix[i][j]


In [9]:
# размер данных
data_value = 1000

# размер элемента данных
matrix_len=7

# заполняем нулями матрицу для данных
data = np.zeros(((data_value+1),(matrix_len*matrix_len+1)))
number = 0

def saveObject(row, matrix_len):
    dist={}
    numberRow = 0
    for i in range(0, len(row)-1, matrix_len ):
            st=[]
            for j in range(i,i+matrix_len):
                index=row[j]
                st.append(index)
            dist.update({numberRow:st})
            numberRow+=1
    dist.update({'answer':'X' if row[len(row)-1] == 1  else '0'})
    return dist
# СОхранение данных в json-файл
def saveInFile(fullMatrix, matrix_len):
    dist={}
    for i in range(len(fullMatrix)):
        index=fullMatrix[i]
        obj = saveObject(index, matrix_len)
        dist.update({i:obj})
    try:
        with open("data_file.json", "w") as write_file:
            json.dump(dist, write_file, indent=4)
        return True
    except ValueError: 
        return False
    

# заполняем матрицу данных элементами 0/1
for h in range(data_value):
    a = createXO()

    # выбираем элемнт случайным образом
    choose = random.randint(0,1)

    if choose == 0:
        m=a.createO()
        # в матрицу данных 0/1 кладем по строкам -> одной строкой один элемент
        for l in range(matrix_len):
            for p in range(matrix_len):
                data[h][p + matrix_len*l] = m[l][p]
                data[h][matrix_len*matrix_len]=choose

    if choose == 1:
        m=a.createX()
        for l in range(matrix_len):
            for p in range(matrix_len):
                data[h][p + matrix_len*l] = m[l][p]
                data[h][matrix_len*matrix_len]=choose
saveInFile(data, matrix_len)

True

In [10]:
import random
def printXO(data):
    for j in range(0, matrix_len):
        for x in data[j * matrix_len: (j + 1) * matrix_len]:
            print(colored(str(int(x)), 'red'), end=' ') if x == 1 else print(int(x), end=' ')
        print()

i = random.randint(0, len(data))
printXO(data[i])

0 0 0 0 0 0 0 
0 0 0 0 1 0 0 
0 1 0 1 0 0 0 
0 0 1 0 0 0 0 
0 1 0 1 0 0 0 
0 0 0 0 1 0 0 
0 0 0 0 0 0 0 


In [11]:

def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
   # перемножая веса на вход, получаем выход
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Функция активации
def activation(x):
    return round((1 + math.exp(-x))**(-1))


# row-данные по крестику/нолику
# предсказываем результат определения элемента, на вход элемент и используемые веса
def a_x(row, weights):
    sum_wx = 0
    for i in range(1,len(weights)):
        sum_wx += weights[i]*row[i]
    return activation(sum_wx - weights[0])    
def z_elem(a,y):
    return (a - y) ** 2
    
def sum_z_elem(train, weights):
    Q=0
    for i in range(len(train)):
        y_i=train[i][matrix_len*matrix_len]
        a = a_x(train[i], weights)
        Q += z_elem(a, y_i)
    return Q

def calc_phi_der(x):
    return math.exp(-x)/((1 + math.exp(-x))**2)
    

In [12]:

# обучение нейрона
def train_weights(train, l_rate, n_epoch):
    
    # иициализируем веса в начале нулями
    weights = [0.0 for i in range(len(train[0]))]
#     Инициализация функции качества
    Q= sum_z_elem(train, weights)
    
    Q1=0
    l=len(train)
    print('L', l)
    for i in range(data_value):
        Q+=predict(train[i], weights)
    n_current = 0
#     while(abs(Q-Q1) > 1e-10 and n_current < n_epoch):
    while( Q!=Q1 and n_current < n_epoch):
        Q1=Q
        n_current +=1
        obj=random.randint(0, len(train)-1)
#         Выходное значение а и eps
        a = a_x(train[obj], weights)
        eps = z_elem(a, train[obj][matrix_len*matrix_len])
        
        scalar_x_w = 0
        
        for i in range(len(weights)):
            scalar_x_w += train[obj][i]*weights[i]
            
        m = 2*(a - train[obj][matrix_len*matrix_len])*calc_phi_der(scalar_x_w)
        
        for j in range(len(weights)):
            weights[j] -= l_rate*m*train[obj][j]
        Q=  Q*(l-1)/l +eps*eps/l        
    return weights

ready_weights= train_weights(data, 0.5, 500)
print(ready_weights)
# Проверка полноты
trueAnswer=0
lenObj=len(data[0])
for i in range(0,1000):
    if(a_x(data[i], ready_weights)==data[i][lenObj-1]):
        trueAnswer+=1
        
print('Точность', trueAnswer/1000)


L 1001
[-0.18971496697700754, -0.18971496697700754, -0.18971496697700754, -0.18971496697700754, -0.18971496697700754, 0.0, 0.0, -0.18971496697700754, 0.0, 0.008345816123193184, -0.23173946256416955, -0.17306258508954214, -0.23173946256416955, 0.0, -0.18971496697700754, 0.24839184445163495, -0.19963445632598353, 0.27058212937718373, -0.15819205803522043, 0.022190284925548753, -0.23173946256416955, -0.18971496697700754, 0.03152290894178711, 0.0166523818874654, 0.4994179027036011, 0.05037031171035519, 0.03210500623818602, -0.23173946256416955, -0.18971496697700754, -0.15921811628718657, -0.172036526837576, 0.2987621561619902, 0.06028503302299246, 0.022190284925548753, -0.23173946256416955, 0.0, 0.03152290894178711, 0.24008527868736274, 0.26767844013943154, 0.0166523818874654, -0.19963445632598353, -0.23173946256416955, 0.0, 0.0, -0.21789499376181398, -0.21789499376181398, 0.03152290894178711, 0.0, 0.0, 0.9878950258425988]
Точность 1.0
